In [1]:
import pandas as pd
import numpy as np
from numpy import log2 as log
error_handler = np.finfo(float).eps

In [40]:
import os
file_path = r'C:\Users\hp\CSE 3210\Lab 2\ecoli-0_vs_1.csv'
file_name = os.path.basename(file_path)
name = (os.path.basename(file_path).split(".")[0]) # gets the first index
file_name

'ecoli-0_vs_1.csv'

In [41]:
data = pd.read_csv(file_name)
data

,0.49,0.29,0.48,0.50,0.56,0.24,0.35,positive
0,0.07,0.40,0.48,0.5,0.54,0.35,0.44,positive
1,0.56,0.40,0.48,0.5,0.49,0.37,0.46,positive
2,0.59,0.49,0.48,0.5,0.52,0.45,0.36,positive
3,0.23,0.32,0.48,0.5,0.55,0.25,0.35,positive
4,0.67,0.39,0.48,0.5,0.36,0.38,0.46,positive
...,...,...,...,...,...,...,...,...
214,0.10,0.49,0.48,0.5,0.41,0.67,0.21,negative
215,0.30,0.51,0.48,0.5,0.42,0.61,0.34,negative
216,0.61,0.47,0.48,0.5,0.00,0.80,0.32,negative
217,0.63,0.75,0.48,0.5,0.64,0.73,0.66,negative


Entropy of Class

In [42]:
def class_entropy(data):
    en_class = 0
    className = data.columns[-1] # last column is class
    patterns = data[className].unique() # yes or no
    for pattern in patterns:
        quotient = data[className].value_counts()[pattern] / data[className].shape[0] # probability
        en_class += -quotient*log(quotient) # equation to determine entropy

    return en_class 

In [43]:
class_entropy(data)

1.031269889536158

Entropy of Features

In [44]:
def feature_entropy(data, feat):
    className = data.columns[-1] # last column is class
    patterns = data[className].unique() # yes or no
    instances = data[feat].unique() # unique value of features
    en_feat = 0
    for ins in instances:
        en_each = 0
        for pattern in patterns:
            nr = len(data[feat][data[feat] == ins][data[className] == pattern])  # matching with class / conditional probability
            dr = len(data[feat][data[feat] == ins])
            quotient = nr/(dr + error_handler)
            en_each += -quotient*log(quotient + error_handler)   # entropy equation
        q2 = dr / data.shape[0]
        en_feat += -q2 * en_each
    return abs(en_feat)

In [ ]:
feature_entropy(data, 'Age')

In [46]:
def root_node(data):
    feat_entropy = []
    ig = []
    for feat in data.columns[:-1]:
        ig.append(class_entropy(data) - feature_entropy(data, feat))  # information gain calculation
    return data.columns[:-1][np.argmax(ig)]   # find max information gain

In [47]:
root_node(data)

' 0.24'

In [48]:
def generate_subtable(data, root, value):
    return data[data[root] == value].reset_index(drop = True)   # create subtable with resetting index

In [ ]:
generate_subtable(data, 'Age', '>55')

In [49]:
def tree_Building(data, tree = None):
    className = data.columns[-1]  # last column is class
    
    root = root_node(data) # getting root node
    
    featValue = np.unique(data[root]) # unique values of root node
    
    if tree is None: # create an empty tree
        tree = {}
        tree[root] = {}
        
    for value in featValue:
        subtable = generate_subtable(data, root, value)  # for each unique value create subtables
        col, counts = np.unique(subtable[className], return_counts = True) # Checking which attribute matches with class
        
        if len(counts) == 1:   # if there is only one such attribute that matches with class
            tree[root][value] = col[0]
        else:
            tree[root][value] = tree_Building(subtable) # else explore that node
            
    return tree

In [50]:
tree = tree_Building(data)
import pprint
pprint.pprint(tree)

{' 0.24': {0.03: ' positive',
           0.05: ' positive',
           0.06: ' positive',
           0.11: ' positive',
           0.14: ' positive',
           0.15: ' positive',
           0.16: ' positive',
           0.17: ' positive',
           0.18: ' positive',
           0.19: ' positive',
           0.2: ' positive',
           0.21: ' positive',
           0.22: ' positive',
           0.23: ' positive',
           0.24: ' positive',
           0.25: ' positive',
           0.26: ' positive',
           0.27: ' positive',
           0.28: ' positive',
           0.29: ' positive',
           0.3: ' positive',
           0.31: ' positive',
           0.32: ' positive',
           0.33: ' positive',
           0.34: ' positive',
           0.35: {' 0.29': {0.28: ' positive',
                            0.29: ' positive',
                            0.33: ' positive',
                            0.36: ' positive',
                            0.37: ' positive',
                 

In [51]:
import pydot

def draw(parent, child):
    edge = pydot.Edge(parent, child)
    graph.add_edge(edge)

def visit(dic, parent = None):
    for key, value in dic.items():
        if isinstance(value, dict): # if value is a dictionary
            if parent:  # if parent node
                draw(parent, str(key)) # draw an edge from parent node to feature
            visit(value, str(key)) # recursively create subtree
        else:
            draw(parent, str(key)) # if value is not a dictionary
            draw(str(key), value)  # then draw a edge

graph = pydot.Dot(graph_type='graph')
visit(tree)
graph.write_png(name + '.png')